In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import torch
import numpy as np
os.environ['TORCH'] = torch.__version__
os.environ['DGLBACKEND'] = "pytorch"
import dgl
import dgl.graphbolt as gb
import torch.nn as nn
import torch.nn.functional as F
import sklearn.metrics
device = torch.device("cpu")

Link Prediction

In [2]:
dataset = gb.BuiltinDataset("cora",root="../../../data/datasets").load()

../../../data/datasets\cora-seeds.zip:   0%|          | 0.00/242k [00:00<?, ?B/s]

Extracting file to ../../../data/datasets
Start to preprocess the on-disk dataset.
Finish preprocessing the on-disk dataset.


In [4]:
graph = dataset.graph
features = dataset.feature 
train_set= dataset.tasks[1].train_set
validation_set= dataset.tasks[1].validation_set
test_set= dataset.tasks[1].test_set
task_name = dataset.tasks[1].metadata["name"]
print(f"task name is {task_name}")


task name is link_prediction


In [17]:
from functools import partial
datapipe = gb.ItemSampler(train_set, batch_size=256, shuffle=True)
datapipe = datapipe.sample_uniform_negative(graph, 5)
datapipe = datapipe.sample_neighbor(graph, [5, 5, 5])
datapipe = datapipe.transform(partial(gb.exclude_seed_edges, include_reverse_edges=True))
datapipe = datapipe.fetch_feature(features, node_feature_keys=["feat"])
datapipe = datapipe.copy_to(device)
train_dataloader = gb.DataLoader(datapipe)

In [18]:
data = next(iter(train_dataloader))
print(f"MiniBatch: {data}")

MiniBatch: MiniBatch(seeds=tensor([[ 156,  817],
                        [ 915, 2597],
                        [2169,  683],
                        ...,
                        [ 148,  905],
                        [ 148,  279],
                        [ 148, 1273]], dtype=torch.int32),
          sampled_subgraphs=[SampledSubgraphImpl(sampled_csc=CSCFormatBase(indptr=tensor([   0,    4,    7,  ..., 7643, 7644, 7646], dtype=torch.int32),
                                                                         indices=tensor([ 931, 1284,  258,  ..., 2242, 1470, 2249], dtype=torch.int32),
                                                           ),
                                               original_row_node_ids=tensor([ 156,  817,  915,  ...,  229, 1053, 1559], dtype=torch.int32),
                                               original_edge_ids=None,
                                               original_column_node_ids=tensor([ 156,  817,  915,  ..., 2621, 2264, 2128], dtype=torc

Defining Model for Node Representation

In [19]:
import dgl.nn as dgln
class SAGE(nn.Module):
    def __init__(self, in_size,hidden_size):
        super().__init__()
        self.layers = nn.ModuleList()
        self.layers.append(dgln.SAGEConv(in_size,hidden_size,"mean"))
        self.layers.append(dgln.SAGEConv(hidden_size,hidden_size,"mean"))
        self.hidden_size = hidden_size
        self.prdictor = nn.Sequential(
            nn.Linear(in_size,hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,1)
        )
    def froward(self,blocks,x):
        hidden_x =x
        for layer_idx,(layer,block) in enumerate(zip(self.layers,blocks)):
            hidden_x = layer(block,hidden_x)
            is_last_layer = layer_idx == len(self.layers)-1
            if not is_last_layer:
                hidden_x =F.relu(hidden_x)
        return hidden_x 


        

